### Flight data extraction and recommendation assignment using agent

In [ ]:
# Dummy data for testing agent
flights_web_page = [
    {
        "flight_number": "SFO-AK123",
        "price": 350,
        "origin": "SFO",
        "destination": "ANC",
        "date": "2025-01-10",
        "flight_duration": 240,
        "layover_duration": 30,
        "group_type": "family",
        "visa_cost": 50,
        "transit_cost": 20,
        "total_duration": 270
    },
    {
        "flight_number": "SFO-AK456",
        "price": 370,
        "origin": "SFO",
        "destination": "FAI",
        "date": "2025-01-10",
        "flight_duration": 210,
        "layover_duration": 60,
        "group_type": "business",
        "visa_cost": 60,
        "transit_cost": 25,
        "total_duration": 270
    },
    {
        "flight_number": "SFO-AK789",
        "price": 400,
        "origin": "SFO",
        "destination": "ANC",
        "date": "2025-01-13",
        "flight_duration": 195,
        "layover_duration": 90,
        "group_type": "solo",
        "visa_cost": 40,
        "transit_cost": 30,
        "total_duration": 285
    },
    {
        "flight_number": "SFO-AK789",
        "price": 200,
        "origin": "SFO",
        "destination": "ANC",
        "date": "2025-01-11",
        "flight_duration": 195,
        "layover_duration": 90,
        "group_type": "solo",
        "visa_cost": 40,
        "transit_cost": 30,
        "total_duration": 285
    },
    {
        "flight_number": "NYC-LA101",
        "price": 250,
        "origin": "SFO",
        "destination": "ANC",
        "date": "2025-01-10",
        "flight_duration": 225,
        "layover_duration": 0,
        "group_type": "family",
        "visa_cost": 55,
        "transit_cost": 15,
        "total_duration": 225
    },
    {
        "flight_number": "CHI-MIA202",
        "price": 200,
        "origin": "ORD",
        "destination": "MIA",
        "date": "2025-01-12",
        "flight_duration": 180,
        "layover_duration": 20,
        "group_type": "business",
        "visa_cost": 0,
        "transit_cost": 10,
        "total_duration": 200
    },
    {
        "flight_number": "BOS-SEA303",
        "price": 120,
        "origin": "BOS",
        "destination": "ANC",
        "date": "2025-01-12",
        "flight_duration": 300,
        "layover_duration": 90,
        "group_type": "solo",
        "visa_cost": 70,
        "transit_cost": 35,
        "total_duration": 390
    },
    {
        "flight_number": "DFW-DEN404",
        "price": 150,
        "origin": "DFW",
        "destination": "DEN",
        "date": "2025-01-10",
        "flight_duration": 150,
        "layover_duration": 0,
        "group_type": "family",
        "visa_cost": 20,
        "transit_cost": 10,
        "total_duration": 150
    },
    {
        "flight_number": "ATL-HOU505",
        "price": 180,
        "origin": "ATL",
        "destination": "IAH",
        "date": "2025-01-10",
        "flight_duration": 130,
        "layover_duration": 30,
        "group_type": "business",
        "visa_cost": 35,
        "transit_cost": 15,
        "total_duration": 160
    }
]


In [ ]:
from dataclasses import dataclass
import json
import datetime
from typing import List, Optional

from pydantic import BaseModel
from pydantic_ai import Agent

export GROQ_API_KEY='your-api-key'

# === Models ===

class FlightDetails(BaseModel):
    flight_number: str
    price: int
    origin: str
    destination: str
    date: str  # Will be converted to datetime.date later
    flight_duration: Optional[int] = None  # in minutes
    layover_duration: Optional[int] = None  # in minutes
    total_duration: Optional[int] = None  # fallback if no breakdown


@dataclass
class Deps:
    web_page_text: str
    req_origin: str
    req_destination: str
    req_date: datetime.date
    flex_days: int = 0
    max_price: Optional[int] = None
    group_type: Optional[str] = None  # e.g., "business", "family", "budget"
    visa_cost: Optional[int] = 0
    transit_cost: Optional[int] = 0


# === Extraction Agent ===

extraction_agent = Agent(
    'groq:gemma2-9b-it',
    output_type=List[FlightDetails],
    system_prompt='Extract all the flight details from the given text.',
)


# === Helper Functions ===

def compute_convenience_score(flight: FlightDetails, deps: Deps) -> float:
    """Compute a score based on travel time, cost, and group preferences."""
    score = 100

    total_time = (
        flight.total_duration or
        (flight.flight_duration or 0) + (flight.layover_duration or 0)
    )
    score -= total_time / 10
    score -= flight.price / 50

    # Adjust for group type
    if deps.group_type == 'business' and (flight.layover_duration or 0) > 60:
        score -= 10
    elif deps.group_type == 'family' and (flight.layover_duration or 0) > 120:
        score -= 15

    return round(score, 2)


def flight_summary(flight: FlightDetails, deps: Deps) -> str:
    """Return a human-readable summary of the flight."""
    total_price = flight.price + deps.transit_cost + deps.visa_cost
    total_time = (
        flight.total_duration or
        (flight.flight_duration or 0) + (flight.layover_duration or 0)
    )
    score = compute_convenience_score(flight, deps)

    return f"""
            ✈️ **Flight {flight.flight_number}** — {flight.origin} → {flight.destination} on {flight.date}
            - 💰 Flight Cost: ${flight.price}
            - 🚇 Transit Cost: ${deps.transit_cost}
            - 🛂 Visa Cost: ${deps.visa_cost}
            - 🧮 Total Cost: ${total_price}
            - ⏱️ Flight Duration: {flight.flight_duration or '?'} mins
            - 🛑 Layover: {flight.layover_duration or 0} mins
            - 🕒 Total Travel Time: {total_time} mins
            - 📊 Convenience Score: {score}/100
            """


# === Core Logic ===

async def extract_flight_matches(deps: Deps) -> List[FlightDetails]:
    """Extract and filter flight options based on user preferences."""
    web_page_text_json = json.dumps(deps.web_page_text)
    extraction_result = await extraction_agent.run(web_page_text_json)

    try:
        flight_data = (
            json.loads(extraction_result.output)
            if isinstance(extraction_result.output, str)
            else extraction_result.output
        )

        all_flights = []
        for item in flight_data:
            item['date'] = datetime.date.fromisoformat(item['date']) \
                if isinstance(item['date'], str) else item['date']
            all_flights.append(FlightDetails(**item))

    except Exception as e:
        print(f"❌ Failed to process flight data: {e}")
        return []

    # Apply filters
    filtered_flights = [
        f for f in all_flights
        if f.origin == deps.req_origin
        and f.destination == deps.req_destination
        and abs((f.date - deps.req_date).days) <= deps.flex_days
        and (deps.max_price is None or f.price <= deps.max_price)
    ]

    # Score and return top 5
    return sorted(
        filtered_flights,
        key=lambda f: compute_convenience_score(f, deps),
        reverse=True
    )[:5]


# === Example Runner ===

async def main():
    deps = Deps(
        web_page_text=flights_web_page,  # Replace or inject `flights_web_page`
        req_origin='SFO',
        req_destination='ANC',
        req_date=datetime.date(2025, 1, 10),
        flex_days=3,
        max_price=600,
        group_type='family',
        visa_cost=50,
        transit_cost=30,
    )

    flights = await extract_flight_matches(deps)

    if not flights:
        print("No matching flights found.")
    else:
        print("\n🧭 Top Flight Options:\n")
        for flight in flights:
            print(flight_summary(flight, deps))


# To execute in an async context
await main()



🧭 Top Flight Options:


✈️ **Flight NYC-LA101** — SFO → ANC on 2025-01-10
- 💰 Flight Cost: $250
- 🚇 Transit Cost: $30
- 🛂 Visa Cost: $50
- 🧮 Total Cost: $330
- ⏱️ Flight Duration: 225 mins
- 🛑 Layover: 0 mins
- 🕒 Total Travel Time: 225 mins
- 📊 Convenience Score: 72.5/100


✈️ **Flight SFO-AK789** — SFO → ANC on 2025-01-11
- 💰 Flight Cost: $200
- 🚇 Transit Cost: $30
- 🛂 Visa Cost: $50
- 🧮 Total Cost: $280
- ⏱️ Flight Duration: 195 mins
- 🛑 Layover: 90 mins
- 🕒 Total Travel Time: 285 mins
- 📊 Convenience Score: 67.5/100


✈️ **Flight SFO-AK123** — SFO → ANC on 2025-01-10
- 💰 Flight Cost: $350
- 🚇 Transit Cost: $30
- 🛂 Visa Cost: $50
- 🧮 Total Cost: $430
- ⏱️ Flight Duration: 240 mins
- 🛑 Layover: 30 mins
- 🕒 Total Travel Time: 270 mins
- 📊 Convenience Score: 66.0/100


✈️ **Flight SFO-AK789** — SFO → ANC on 2025-01-13
- 💰 Flight Cost: $400
- 🚇 Transit Cost: $30
- 🛂 Visa Cost: $50
- 🧮 Total Cost: $480
- ⏱️ Flight Duration: 195 mins
- 🛑 Layover: 90 mins
- 🕒 Total Travel Time: 285 mins
- 📊 Convenience Score: 63.5/100